In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [12]:
# Cargar los datos
data = np.loadtxt('/content/drive/MyDrive/laboratorios_IA/lab_5/maleModifiers.csv', delimiter=',', skiprows=1)
X = data[:, :-1] / 255.0  # Normalizar los valores de píxeles
y = data[:, -1]
print(data.shape)

(15189, 785)


In [4]:
# Establecer el dígito 10 en 0, ya que los datos de MATLAB asignan 10 para el dígito 0
y[y == 11] = 0

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir los datos de numpy a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [5]:
# Crear conjuntos de datos y cargadores de datos
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64)

In [6]:
# Definir la arquitectura de la red neuronal
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [7]:
# Definir hiperparámetros
input_size = 28 * 28
hidden_size = 100
num_classes = 10
learning_rate = 0.001
num_epochs = 10


In [8]:
# Inicializar modelo, función de pérdida y optimizador
model = NeuralNetwork(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Entrenamiento del modelo
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Propagación hacia adelante
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Retropropagación y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/190], Loss: 0.7660
Epoch [2/10], Step [100/190], Loss: 0.2688
Epoch [3/10], Step [100/190], Loss: 0.2001
Epoch [4/10], Step [100/190], Loss: 0.3086
Epoch [5/10], Step [100/190], Loss: 0.3026
Epoch [6/10], Step [100/190], Loss: 0.2360
Epoch [7/10], Step [100/190], Loss: 0.2466
Epoch [8/10], Step [100/190], Loss: 0.1255
Epoch [9/10], Step [100/190], Loss: 0.0886
Epoch [10/10], Step [100/190], Loss: 0.0865


In [10]:
# Evaluación del modelo
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Precisión del modelo en el conjunto de prueba: {} %'.format(100 * correct / total))



Precisión del modelo en el conjunto de prueba: 93.61421988150099 %


una precisión del 93.61% es un resultado sólido que sugiere que nuestra red neuronal está aprendiendo efectivamente de los datos y haciendo predicciones útiles. 
las capas ocultas en una red neuronal pueden influir significativamente en su capacidad para aprender y generalizar a partir de los datos